# 时间序列模型筛选过程示例

## 1. 导入依赖库
模型测试结果将存储在项目根路径`results.db`中。

In [1]:
import sqlite3

## 2. 连接数据库

使用`sqlite3.connect`方法连接数据库，若数据库未事先创建，代码将自动创建空数据库。

In [2]:
conn = sqlite3.connect('results.db')

## 3. 创建游标对象
连接数据库后，使用`cursor()`方法创建游标对象，为后续执行SQL命令做准备。

In [3]:
cursor = conn.cursor()

## 4. 创建数据库表
使用`cursor.execute()`方法在python中执行`sqlite`命令。此处创建了一个名为`Results`的数据表，表中数据条目如下：
- `id`：数据记录序号，整数类型。是数据表主键。
- `model_name`：测试模型名称，文本类型。
- `task_type`：测试任务类型名称，文本类型。测试任务类型限制为五种主要的任务之一（短期预测、长期预测、分类、异常检测、填补）
- `dataset_name `：测试数据集名称，文本类型。
- `seq_len`：测试数据输入长度，整数类型。
- `metric`：测试评估算法，文本类型。
- `value`：测试评估结果，实数类型。


In [4]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Results (
        id INTEGER PRIMARY KEY,
        model_name TEXT,
        task_type TEXT CHECK(task_type IN ('long-term-forecast')),
        dataset_name TEXT,
        seq_len INTEGER,
        metric TEXT,
        value REAL
    )
''')

## 5. 数据构建与数据库写入
数据记录以元组列表形式存储，在下面的代码块中，我们创建了一个名为`data_example`的样例数据。这个数据中包含了TimesNet、Dlinear与Informer在ECL数据长期预测任务上输入长度96设置场景下的MSE结果。我们使用cursor.executemany()方法将包含多条数据的dataexample插入数据库中。


In [ ]:
long_forecast_data = [
    ('DLinear', 'long-term-forecast', 'Gas', 96, 'MSE', 1.637),
    ('DLinear', 'long-term-forecast', 'Gas', 192, 'MSE', 1.683),
    ('DLinear', 'long-term-forecast', 'Gas', 336, 'MSE', 1.658),
    ('DLinear', 'long-term-forecast', 'Gas', 720, 'MSE', 1.618),
    
    ('PatchTST', 'long-term-forecast', 'Gas', 96, 'MSE', 1.750),
    ('PatchTST', 'long-term-forecast', 'Gas', 192, 'MSE', 1.800),
    ('PatchTST', 'long-term-forecast', 'Gas', 336, 'MSE', 1.991),
    ('PatchTST', 'long-term-forecast', 'Gas', 720, 'MSE', 1.981),

    ('iTransformer', 'long-term-forecast', 'Gas', 96, 'MSE', 1.706),
    ('iTransformer', 'long-term-forecast', 'Gas', 192, 'MSE', 1.772),
    ('iTransformer', 'long-term-forecast', 'Gas', 336, 'MSE', 1.970),
    ('iTransformer', 'long-term-forecast', 'Gas', 720, 'MSE', 2.025),
]

cursor.executemany('''
    INSERT INTO Results (model_name, task_type, dataset_name, seq_len, metric, value)
    VALUES (?, ?, ?, ?, ?, ?)
''', long_forecast_data)

conn.commit()
cursor.close()
conn.close()


以上数据仅为用于演示的小部分数据，若需要导入全部预置数据库记录，请运行脚本./jupyter/sql.py快速导入。

## 时序模型算法筛选
使用函数`model_filtrate()`使用评测结果对模型进行筛选。函数参数如下：
- `model_name`：模型名称，用于寻找与指定模型名相同的数据。
- `task_type`：任务类型名称，用于寻找与指定任务类型相同的数据。
- `dataset_name`：测试数据集名称，用于寻找与指定测试数据集相同的数据。
- `seq_len`：模型输入长度，用于寻找与指定模型输入长度相同的数据。
- `metric`：模型评测算法，用于寻找与指定模型评测算法相同的数据。
- `orderBy`：返回结果排序。

In [1]:
from sql import model_filter
res = model_filter(model_name=('TimesNet','Timer','Moirai'), dataset_name='ECL', metric='MSE', seq_len=96)
for row in res:
    print(row)

Connection established to ---> results.db
Reading from table name ---> Results
sql input command:  SELECT distinct model_name, value FROM Results  WHERE 1 AND model_name IN('TimesNet', 'Timer', 'Moirai') AND dataset_name='ECL' AND seq_len='96' AND metric='MSE'
('TimesNet', 0.168)
('Timer', 0.129)
('Moirai', 0.13)
